# Let's Start Question Answering

因為pytorch-transformers 在1.0.0版有許多的bug 所以我們這邊安裝master branch來使用

In [0]:
!pip install https://github.com/huggingface/transformers/releases/download/v0.1.2/pytorch_pretrained_bert-0.1.2.tar.gz

     |████████████████████████████████| 51kB 621kB/s 
  Created wheel for pytorch-pretrained-bert: filename=pytorch_pretrained_bert-0.1.2-cp36-none-any.whl size=32311 sha256=223761a9734eee4f58634046dc0263909e67146247606708ae84dd3178a2a4ba
  Stored in directory: /root/.cache/pip/wheels/95/f2/55/b2c19f6d3a9bbede0fb2d8874b33510e5fe96f33a8549166c9
Successfully built pytorch-pretrained-bert


In [0]:
!nvidia-smi
!rm -rf bert_chinese*
!rm -rf pytorch_pretrained_bert*
!wget -q --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1SXz4Hr073OCuIOqzmi1Ool6m9-g8yYOf' -O bert_chinese.zip
!unzip -q -o bert_chinese.zip
!wget -q --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=15vkVWzKDlik_l2MAkvGI6tIhPauQiGaW' -O pytorch_pretrained_bert.zip
!unzip -q -o pytorch_pretrained_bert.zip
# !wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1IIVjgGX_JeXBNQv-vRAj1JLFvv2x9EVu' -O pytorch_pretrained_bert-0.1.2.zip
# !unzip pytorch_pretrained_bert-0.1.2.zip
# %cd pytorch_pretrained_bert-0.1.2
# !python setup.py install
# %cd ..

Fri Oct  4 15:46:55 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import numpy as np
from tqdm import tqdm


def to_list(tensor):
    return tensor.detach().cpu().tolist()
 

def _get_best_indexes(logits, n_best_size=1):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes
 
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)
    
def predict(eval_dataloader, model, tokenizer, device='cpu'):


    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions = []
    for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc='Iteration'):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        
        with torch.no_grad():
            print(input_ids.shape)
#             tmp_eval_loss, logits = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask, label_ids)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        
        logits = softmax(logits.tolist())
        for logit in logits:
            predictions.append(logit)

    return np.argmax(predictions[0])
  

In [0]:
import collections

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

        
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
            
def convert_examples_to_features(context, question, choices, tokenizer, ans, max_seq_length=300, label_list=[1,2,3,4]):
    
    text_bs = list()
    for choice in choices:
        # text_b = tokenization.convert_to_unicode(str(row['choice%d' % i]))
        text_bs.append("{} {}".format(question, choice))
        #text_b.append(choice)
      

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i    

    multiple_choice_input_ids = []
    multiple_choice_input_masks = []
    multiple_choice_segment_ids = []
    for text_b in text_bs:  
    

        tokens_a = tokenizer.tokenize(context)  

        tokens_b = None
        if text_b:
            tokens_b = tokenizer.tokenize(text_b)   

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)] 

        
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)   

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)   

        input_ids = tokenizer.convert_tokens_to_ids(tokens) 

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)   

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)   

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        multiple_choice_input_ids.append(input_ids)
        multiple_choice_input_masks.append(input_mask)
        multiple_choice_segment_ids.append(segment_ids)
    assert len(multiple_choice_input_ids) == 4
    assert len(multiple_choice_input_masks) == 4
    assert len(multiple_choice_segment_ids) == 4    

    label_id = label_map[ans] 
    
    

    f = InputFeatures(input_ids=multiple_choice_input_ids,
                      input_mask=multiple_choice_input_masks,
                      segment_ids=multiple_choice_segment_ids,
                      label_id=label_id)  
    
    

    all_input_ids = torch.tensor([f.input_ids], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    eval_dataloader = DataLoader(eval_data, sampler=SequentialSampler(eval_data), batch_size=1)

    return eval_dataloader, tokens




In [0]:
# 因為要我們今天要跑的是中文QA 所以只有Bert可以用
import torch
from pytorch_pretrained_bert import (BertConfig, BertForSequenceClassification,
                                 BertTokenizer)

device = torch.device("cuda")
 
model_name = 'bert_chinese'
config_class, tokenizer_class = BertConfig, BertTokenizer
# model = model_class.from_pretrained('bert_chinese').to(device)
model = torch.load(model_name + '/model.cpt') 
tokenizer = torch.load(model_name + '/tokenizer.cpt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'pytorch_pretrained_bert.modeling.BertForSequenceClassification' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the obj

In [0]:
context = '關於芒果品種的小故事，土芒果是最早來台灣的芒果，是荷蘭人在台期間引進，同時也是產期最早的芒果，早生品種在4月即可採收，盛產期5至7月，果皮黃綠色，纖維較粗，酸甜有味，是許多人兒時記憶中的味道；愛文則是台灣栽培最多的主力品種，1954年由美國佛羅里達引進，產期5至7月，外皮鮮紅討喜，甜酸適中符合現代大眾的喜好；凱特同樣與愛文同時引進，但較晚生成，產期可延遲至9至10月，故有「九月檨」之稱號，果實外觀黃色，為較大之卵圓型，果肉多汁而帶微酸。'
question = '請問台灣的主力芒果品種為何？'
choices = ['小芒果', '土芒果', '愛文芒果', '凱特芒果']
label = 3
data, tokens = convert_examples_to_features(context, question, choices, tokenizer, 2)
pred = predict(data, model, tokenizer, device=device)

print('\n\n', context)
print(question)
print('Predicted answer: ', choices[pred])
print('Ground truth: ', choices[label-1])

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration: 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]



 關於芒果品種的小故事，土芒果是最早來台灣的芒果，是荷蘭人在台期間引進，同時也是產期最早的芒果，早生品種在4月即可採收，盛產期5至7月，果皮黃綠色，纖維較粗，酸甜有味，是許多人兒時記憶中的味道；愛文則是台灣栽培最多的主力品種，1954年由美國佛羅里達引進，產期5至7月，外皮鮮紅討喜，甜酸適中符合現代大眾的喜好；凱特同樣與愛文同時引進，但較晚生成，產期可延遲至9至10月，故有「九月檨」之稱號，果實外觀黃色，為較大之卵圓型，果肉多汁而帶微酸。
請問台灣的主力芒果品種為何？
Predicted answer:  愛文芒果
Ground truth:  愛文芒果


In [0]:

question = '土芒果是哪國人引進的？'#'請問何者與愛文同時引進?'
choices = ['荷蘭', '台灣', '美國', '日本']
label = 1
data, tokens = convert_examples_to_features(context, question, choices, tokenizer, 4)
pred = predict(data, model, tokenizer, device=device)

print('\n\n', context)
print(question)
print('Predicted answer: ', choices[pred])
print('Ground truth: ', choices[label-1])

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration: 100%|██████████| 1/1 [00:00<00:00,  3.46it/s]



 關於芒果品種的小故事，土芒果是最早來台灣的芒果，是荷蘭人在台期間引進，同時也是產期最早的芒果，早生品種在4月即可採收，盛產期5至7月，果皮黃綠色，纖維較粗，酸甜有味，是許多人兒時記憶中的味道；愛文則是台灣栽培最多的主力品種，1954年由美國佛羅里達引進，產期5至7月，外皮鮮紅討喜，甜酸適中符合現代大眾的喜好；凱特同樣與愛文同時引進，但較晚生成，產期可延遲至9至10月，故有「九月檨」之稱號，果實外觀黃色，為較大之卵圓型，果肉多汁而帶微酸。
土芒果是哪國人引進的？
Predicted answer:  荷蘭
Ground truth:  荷蘭
